In [1]:
from os import path, listdir
import sys
sys.path.append("..")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.tracking import *

In [2]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory('../DB/test', batch_size = 3568, class_mode = 'categorical', target_size = (227, 227))

# On récupère les images d'un batch de la taille du dataset
i = 0
for image, lab in test_generator:
    if i == 0: break # sinon le generator continue sa boucle (??)

Found 3568 images belonging to 5 classes.


# Tracking vidéo
En partant d'une première détection de l'objet d'intérêt qui prend la forme d'une boite englobante que l'on suppose correcte, on va suivre l'objet sur les frames suivantes.

Pour chaque frame, on prend la boîte englobante de la frame précédente, on va créer un set de nouvelles boîtes potentielles. On va ensuite récupérer le "patch" de l'image associé à chaque nouvelle boîte englobante et effectuer une prediction à l'aide du modèle précédemment créé. On conservera comme nouvelle boîte celle qui fournira la prédiction la plus précise (pourcentage le plus élevé).

In [3]:
# on charge le modèle
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

In [4]:
model.evaluate(image, lab)

112/112 [==============================] - 34s 300ms/step - loss: 0.0565 - accuracy: 0.9846 - categorical_accuracy: 0.9846 - recall: 0.9835 - precision: 0.9851


[0.056513767689466476,
 0.9845852255821228,
 0.9845852255821228,
 0.9834641218185425,
 0.9851207137107849]

In [34]:
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        tracking(model, path_video, path_boxes)

  0%|          | 0/186 [00:00<?, ?it/s]

Loading 186 frames


 34%|███▍      | 63/186 [00:50<17:12,  8.40s/it] 

KeyboardInterrupt: 

## Move-to-data
Nous implémentons ici la méthode d'apprentissage incrémental "move-to-data" décrite dans _Move-to-Data: A new Continual Learning approach with Deep CNNs, Application for image-class recognition_

Pour chaque nouvelle image, on met à jour les poids de la dernière couche du réseau en appliquant la formule suivante 
$$w_j′=w_j+(||w_j||∗\frac{v_i}{||v_i||} − w_j)*\epsilon$$

In [35]:
def move_to_data(image, model, epsilon, j):
    y_, _ = model(tf.expand_dims(image, axis=0))
    
    # get last layer
    outputs = [layer.output for layer in model.layers]
    last_layer = outputs[-1]
    w, b = last_layer.get_weights()
    
    new_weights = w.copy()
    
    w_j = w[:, j]
    new_w = w_j + (tf.norm(w_j) * (y_ / tf.norm(y_)) - w_j) * epsilon
    new_weights[:, j] = new_w

    last_layer.set_weights([new_weights, b])

In [36]:
# On modifie légèrement le modèle pour avoir les bonnes sorties
output = [layer.output for layer in model.layers]
move_model = tf.keras.Model(output[0], [output[-2], output[-1]])

move_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
    'accuracy', 
    'categorical_accuracy',
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision()])

In [46]:
# On charge toutes les images issues du tracking video
images = []
labels = []
sum_iou = 0
for file in listdir("../GT_pred"):
    if file.endswith(".txt"):
        
        predicted_boxes = read_bounding_box(path.join('..', 'GT_pred', file))
        
        m = re.search('(.+)_pred_box\.txt', file)
        name = m.group(1)
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        
        real_boxes = read_bounding_box(path.join('..', 'GT_test', name + '_2_bboxes.txt'))
        
        cap = cv2.VideoCapture(path_video)
        nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if 'CanOfCocaCola' in file:
            label = 1
        elif 'MilkBottle' in file:
            label = 2
        elif 'Bowl' in file:
            label = 0
        elif 'Rice' in file:
            label = 3
        elif 'Sugar' in file:
            label = 4

        for f in range(nframes):
            p += 1
            ret, frame = cap.read()

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            f_width, f_height, _ = frame.shape

            if f in predicted_boxes:
                x, y, w, h = predicted_boxes[f]
                patch = frame[y:y+h, x:x+w, :]
                images.append(patch)
                labels.append(label)
                
                if f in real_boxes:
                    sum_iou += bb_intersection_over_union(predicted_boxes[f], real_boxes[f])
        
print(f'Mean iou = {sum_iou / p}')

Mean iou = 0.0441307964404913


In [42]:
# On effectue 10 epochs de move to data sur les images mélangeant les images à chaque époch
for i in range(10):
    perm = np.random.permutation(len(images))
    
    for idx in perm:
        im = images[idx]
        if 0 not in im.shape:
            move_to_data(im, move_model, 0.0001, labels[idx])

In [43]:
model.evaluate(image, lab)

112/112 [==============================] - 36s 318ms/step - loss: 0.0856 - accuracy: 0.9723 - categorical_accuracy: 0.9723 - recall: 0.9714 - precision: 0.9758


[0.08563674986362457,
 0.9722533822059631,
 0.9722533822059631,
 0.9714125394821167,
 0.9757882952690125]

## Fine-tuning

In [52]:
# on charge le modèle
model_fine = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

# on laisse le dernier layer entrainable uniquement
for layer in model_fine.layers[:-1]:
    layer.trainable = False

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_fine.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[
    'accuracy',
    'categorical_accuracy',
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision()])

In [53]:
# On effectue 10 epochs de move to data sur les images mélangeant les images à chaque époch
for i in range(5):
    perm = np.random.permutation(len(images))
    
    for idx in perm:
        im = images[idx]
        if 0 not in im.shape:
            #print(tf.expand_dims(im, axis=0).shape)
            im = tf.image.resize(im, (227, 227), method='nearest')
            label = np.zeros((5))
            label[labels[idx]] = 1
            label = tf.expand_dims(label, axis=0)
            model_fine.fit(x=tf.expand_dims(im, axis=0), y=label, epochs=1)

1/1 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 931us/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 1570.4991 - accuracy: 0.0000e+00 - categorical_accuracy: 0.0000e+00 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00


...

1/1 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - categorical_accuracy: 1.0000 - recall_5: 1.0000 - precision_5: 1.0000


In [54]:
model_fine.evaluate(image, lab)  

112/112 [==============================] - 31s 279ms/step - loss: 0.5759 - accuracy: 0.8268 - categorical_accuracy: 0.8268 - recall_5: 0.8072 - precision_5: 0.8453


[0.5759417414665222,
 0.8267937302589417,
 0.8267937302589417,
 0.8071748614311218,
 0.8453184366226196]